In [107]:
import pandas as pd
import xgboost as xgb

In [115]:
train_raw = pd.read_json('../contracts/artifacts/train.json')
# train
# 0 - addr
# 1 - tokens
# 2 - days
# 3 - returned


In [116]:
train = train_raw.groupby(0)[[1,2]].sum()
train = train.rename(columns={1: 'loans_tokens_sum', 2: 'loans_days_sum'})
train.index.rename('id', inplace=True)

In [117]:
train_y = train_raw.groupby(0)[[1]].sum()
train_y = train_y.rename(columns={1: 'trust'})
train_y.index.rename('id', inplace=True)

success = train_raw[train_raw[3]==True].groupby(0)[[1]].sum()
success = success.rename(columns={1: 'success'})
success.index.rename('id', inplace=True)
train_y['trust'] = (success['success'].fillna(0) / train_y['trust']).fillna(0)



In [118]:
train['loans_count'] = train_raw.groupby(0)[1].count()
train['success_loans_count'] = train_raw[train_raw[3]==True].groupby(0)[1].count()
train['avg_tokens'] = train['loans_tokens_sum'] / train['loans_count']
train['avg_days'] = train['loans_days_sum'] / train['loans_count']

train


,loans_tokens_sum,loans_days_sum,loans_count,success_loans_count,avg_tokens,avg_days
id,,,,,,
0x00a90dafa86b37575e505d681d1e63e97816cc9d,132,2,2,2.0,66.000000,1.000000
0x00cf5dd47291b035b126cb42dd65335793befa30,8392,17,7,NaN,1198.857143,2.428571
0x03db1946729adec63e676d428eec56d754a83fd2,6469,18,10,NaN,646.900000,1.800000
0x044301ce5628462cc21e7c0616cfe8d05256e2fa,3115,75,20,20.0,155.750000,3.750000
0x046c84ade7ff7235cc7b5022c5bfd33a5e132078,1127,11,7,7.0,161.000000,1.571429
0x064363e9537ae3f1385dc81e23cfd3db624fda95,3589,27,23,23.0,156.043478,1.173913
0x06a39d81ef8bec3a11700a2c76058e5d70b0189a,6006,40,18,NaN,333.666667,2.222222
0x074507f189e51baca3f87f9317fab245f80b672a,1660,26,10,1.0,166.000000,2.600000
0x0981b3b848020f30067c1c093d6ffc5edff7b6d0,1143,25,12,12.0,95.250000,2.083333


In [119]:
trainDmatrix = xgb.DMatrix(train, label=train_y['trust'])

In [120]:
booster = xgb.train({}, trainDmatrix)
#booster.predict(xgb.DMatrix(train))

In [121]:
booster.save_model('model.bin')

In [122]:
bst = xgb.Booster() 
bst.load_model('model.bin')
bst.predict(xgb.DMatrix(train))

array([0.9815607 , 0.0145438 , 0.0145438 , 0.95452446, 0.9841066 ,
       0.99104065, 0.0145438 , 0.19579902, 0.9700887 , 0.0145438 ,
       0.96301633, 0.94379675, 0.0145438 , 0.9785692 , 0.9773059 ,
       0.0145438 , 0.9794425 , 0.9785964 , 0.42644727, 0.98217905,
       0.0145438 , 0.0145438 , 0.0145438 , 0.0145438 , 0.0145438 ,
       0.0145438 , 0.9757669 , 0.96656907, 0.9746719 , 0.94728225,
       0.9849912 , 0.97132546, 0.72981286, 0.5720914 , 0.95327425,
       0.16564345, 0.87363845, 0.10171387, 0.0145438 , 0.0145438 ,
       0.0145438 , 0.9841399 , 0.0145438 , 0.0145438 , 0.0145438 ,
       0.97779155, 0.0145438 , 0.9737698 , 0.02023533, 0.9841399 ,
       0.9803108 , 0.0145438 , 0.98514557, 0.97779155, 0.9659889 ,
       0.0145438 , 0.7565124 , 0.89369196, 0.0145438 , 0.0145438 ,
       0.9630449 , 0.95797133, 0.9105804 , 0.8460405 , 0.76314515,
       0.9017905 , 0.9777451 , 0.0145438 , 0.0145438 , 0.9758591 ,
       0.9841399 , 0.1360434 , 0.97779155, 0.85398114, 0.01454